In [1]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels
import statsmodels.stats.diagnostic as smd
from statsmodels.formula.api import ols
from statsmodels.compat import lzip
import statsmodels.tools.eval_measures as sme

from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import math
import seaborn as sns
import numpy as np
import pandas as pd
import csv
import import_ipynb
import functions as func

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

importing Jupyter notebook from functions.ipynb


# Fit Model

In [2]:
def fit_ols(predictors, response, data):
    X = data[predictors]
    y = data[response]
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()
    return results

# Predict

In [3]:
def predict_ols(predictors, response, results, data):
    X_test = data[predictors]
    X_test = sm.add_constant(X_test)
    y_pred = results.predict(X_test)
    return y_pred

# Hybrid Stepwise

In [4]:
def stepwise(predictors, response, alpha, data, show=False):
    var_to_fit = []
    pvalues_new = {}
    pvalues_old = {}
    global var_selected
    var_selected = []
    var_removed = []
    var_kept = []

    pvalues_final = {}
    
    
    def clear_values():
        pvalues_new.clear()
        pvalues_old.clear()
        var_removed = []
        var_kept = []    
    
    def fit_model(x):
        pvalues_old.clear()
        X = data[x]
        y = data[response]
        X = sm.add_constant(X)
        model = sm.OLS(y, X)
        results = model.fit()
        for i in range(len(x)-1):
            pvalues_old[x[i]] = results.pvalues[i+1]
        pvalues_new[x[-1]] = results.pvalues[-1]
    
    def check_pvalues_new(val):
        """check if pvalues_new contains p<alpha"""
        for key in pvalues_new:
            if pvalues_new[key] <= val:
                return True
                break
            else:
                continue
        return False

    def check_pvalues_old(val):
        """check if pvalues_old contains p>alpha"""

        for key in pvalues_old:
            if pvalues_old[key] > val:
                return True
                break
            else:
                continue
        return False
    
    clear_values()
    count = 1
    keep_going = True
    while keep_going:
        if show:
            print(f"Round: {count}")
            print(f"Predictors: {predictors}")
        clear_values()

        for i in range(len(predictors)):
            var_to_fit = var_selected + [predictors[i]]
            fit_model(var_to_fit)
        if show:
            print(f"pvalues_new: {pvalues_new}")

        if check_pvalues_new(alpha):
            var_min = min(pvalues_new, key=pvalues_new.get)
            if show:
                print(f"var min: {var_min}")
            var_to_fit = var_selected + [var_min]
            fit_model(var_to_fit)
            if check_pvalues_old(alpha):
                for var in var_selected:
                    if pvalues_old[var] > alpha:
                        var_remove = var
                    else:
                        continue
                    var_selected.remove(var)
                    predictors.append(var)
                    var_removed.append(var)
                var_kept = var_selected
            else:
                var_kept = var_selected
            var_selected.append(var_min)
            predictors.remove(var_min)
            if show:
                print(f"Var kept: {var_kept}")
                print(f"var removed: {var_removed}")
                print(f"var_selected: {var_selected}")
                print("-------------------------------------------------------------------------------------")            
            count += 1
        else:
            X = data[var_selected]
            y = data[response]
            X = sm.add_constant(X)
            model = sm.OLS(y, X)
            results = model.fit()
            for i in range(len(var_selected)):
                pvalues_final[var_selected[i]] = results.pvalues[i+1]
            keep_going = False
            p_dropped = {key : round(pvalues_new[key], 3) for key in pvalues_new}
            if show:
                print(f"P-Values Dropped: {p_dropped}")
    print("----------------------------------------------------")
    print("Stepwise Complete")
    print(f"Total Rounds: {count}")
    print(f"Features Selected: {var_selected}")
    return var_selected

# Model Evaluation with CV

In [5]:
def model_eval(predictors, response, data, num_fold=5):  
    kf = KFold(n_splits=num_fold, random_state=0, shuffle=True)

    mse_list = []
    rmse_list = []
    rmspe_list = []
    meanabs_list = []
    medianabs_list = []
    y_predAll = []
    y_testAll = []
    for train_index, test_index in kf.split(data):
        X_train = data.iloc[train_index][predictors]
        X_train = sm.add_constant(X_train)
        y_train = data.iloc[train_index][response]
        model = sm.OLS(y_train, X_train)
        results = model.fit()

        X_test = data.iloc[test_index][predictors]
        X_test = sm.add_constant(X_test)
        y_pred = results.predict(X_test)
        y_test = data.iloc[test_index][response]

        mse = sme.mse(y_pred, y_test, axis=0)
        rmse = sme.rmse(y_pred, y_test, axis=0)
        rmspe = sme.rmspe(y_pred, y_test, axis=0)
        meanabs = sme.meanabs(y_pred, y_test, axis=0)
        medianabs = sme.medianabs(y_pred, y_test, axis=0)

        mse_list.append(mse)
        rmse_list.append(rmse)
        rmspe_list.append(rmspe)
        meanabs_list.append(meanabs)
        medianabs_list.append(medianabs)
        y_predAll = np.append(y_predAll, y_pred)
        y_testAll = np.append(y_testAll, y_test)
        
    avg_mse = round(sum(mse_list)/len(mse_list), 3)
    avg_rmse = round(sum(rmse_list)/len(rmse_list), 3)
    avg_rmspe = round(sum(rmspe_list)/len(rmspe_list), 3)
    avg_meanabs = round(sum(meanabs_list)/len(meanabs_list), 3)
    avg_medianabs = round(sum(medianabs_list)/len(medianabs_list), 3)

    print(f"MSE: {avg_mse}; RMSE: {avg_rmse}; RMSPE: {avg_rmspe}; MEANABS: {avg_meanabs}; MEDIANABS: {avg_medianabs}")
    
    fig, ax = plt.subplots()
    fig.set_size_inches(5, 5)
    sns.scatterplot(x=y_testAll, y=y_predAll)
    plt.plot([0, 50], [0, 50], linewidth=2, color='red', alpha=0.6)
    ax.set(xlabel='Salary_test', ylabel='Salary_pred')
    plt.show()
    
    model_eval.mse_list = mse_list
    model_eval.rmse_list = rmse_list
    model_eval.rmspe_list = rmspe_list
    model_eval.meanabs_list = meanabs_list
    model_eval.medianabs_list = medianabs_list
    model_eval.y_test = y_testAll
    model_eval.y_pred = y_predAll